In [1]:
import mysql.connector
import pandas as pd

# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='127.0.0.1',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hj010701',   # MySQL 사용자 비밀번호
    database='employee'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()

sql_query = "SELECT * FROM member_based_okr_assignments"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_based_okr_assignments = pd.DataFrame(result, columns=column_names)


sql_query = "SELECT * FROM okr_peer_30"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

okr_df = pd.DataFrame(result, columns=column_names)


sql_query = '''
SELECT *
FROM member_based_okr_assignments
JOIN okr_peer_30 
ON okr_peer_30.OKR_NUM IN (member_based_okr_assignments.project1, member_based_okr_assignments.project2, member_based_okr_assignments.project3);
'''
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_okr = pd.DataFrame(result, columns=column_names)


In [2]:
import pandas as pd
import numpy as np

# CSV 파일 불러오기
df = member_okr

# 'Member' 컬럼 기준으로 오름차순 정렬
df_sorted = df.sort_values(by='Member', ascending=True)

# 데이터를 저장할 리스트 초기화
flattened_data = []

# 전체 열 개수 확인
num_columns = df_sorted.shape[1]

# 데이터 처리 및 열 범위에 따른 조건 설정
# for i in range(100):
#     # 열 범위 설정
#     if 0 <= i < 20:
#         selected_columns = [12] + [col for col in range(13, 19) if col < num_columns]
#     elif 20 <= i < 40:
#         selected_columns = [12] + [col for col in range(19, 25) if col < num_columns]
#     elif 40 <= i < 60:
#         selected_columns = [12] + [col for col in range(25, 31) if col < num_columns]
#     elif 60 <= i < 80:
#         selected_columns = [12] + [col for col in range(31, 37) if col < num_columns]
#     elif 80 <= i < 100:
#         selected_columns = [12] + [col for col in range(37, 43) if col < num_columns]
        

for i in range(50):
    # 열 범위 설정
    if 0 <= i < 10:
        selected_columns = [12] + [col for col in range(13, 19) if col < num_columns]
    elif 10 <= i < 20:
        selected_columns = [12] + [col for col in range(19, 25) if col < num_columns]
    elif 20 <= i < 30:
        selected_columns = [12] + [col for col in range(25, 31) if col < num_columns]
    elif 30 <= i < 40:
        selected_columns = [12] + [col for col in range(31, 37) if col < num_columns]
    elif 40 <= i < 50:
        selected_columns = [12] + [col for col in range(37, 43) if col < num_columns]

    # 선택된 열의 첫 번째 데이터 가져오기
    first_row_data = df_sorted.iloc[3 * i : 3 * (i + 1), selected_columns[0]].T.tolist()

    # 나머지 열 데이터 가져와 병합
    other_data = df_sorted.iloc[3 * i : 3 * (i + 1), selected_columns[1:]].values.flatten().tolist()

    # 데이터 조합
    combined_data = [i + 1, np.nan] + first_row_data + other_data
    flattened_data.append(combined_data)

# 컬럼명 설정 (최대 길이에 맞게 조정)
column_names = [
    'member', 'N_OKR', 'pr1_score', 'pr2_score', 'pr3_score', 
    'pr1_1', 'pr1_2', 'pr1_3', 'pr1_4', 'pr1_5', 'pr1_6', 
    'pr2_1', 'pr2_2', 'pr2_3', 'pr2_4', 'pr2_5', 'pr2_6', 
    'pr3_1', 'pr3_2', 'pr3_3', 'pr3_4', 'pr3_5', 'pr3_6'
]
max_length = max(len(row) for row in flattened_data)
adjusted_column_names = column_names[:max_length]

# 새로운 DataFrame 생성 및 CSV로 저장
data = pd.DataFrame(flattened_data, columns=adjusted_column_names)

data=data.iloc[:,1:]
data

,N_OKR,pr1_score,pr2_score,pr3_score,pr1_1,pr1_2,pr1_3,pr1_4,pr1_5,pr1_6,...,pr2_3,pr2_4,pr2_5,pr2_6,pr3_1,pr3_2,pr3_3,pr3_4,pr3_5,pr3_6
0,NaN,68.17,60.93,60.03,4,5,5,5,3,5,...,5,3,3,2,3,5,2,3,5,3
1,NaN,60.53,67.60,61.60,4,5,4,3,2,5,...,2,5,5,4,3,3,4,5,3,3
2,NaN,65.57,63.13,76.10,5,5,4,3,5,3,...,4,5,5,4,3,3,2,2,4,4
3,NaN,62.17,62.53,65.93,4,4,3,4,5,3,...,5,5,3,5,4,5,5,5,3,5
4,NaN,61.57,64.07,53.77,5,5,4,5,4,5,...,4,5,5,5,3,3,4,4,3,2
5,NaN,56.27,67.20,51.17,2,5,3,2,4,2,...,2,5,4,4,2,2,5,2,3,5
6,NaN,57.03,60.53,65.53,4,4,3,4,4,3,...,3,3,4,3,4,4,3,4,4,2
7,NaN,59.00,67.43,58.87,1,2,4,4,1,3,...,3,2,2,3,4,5,3,4,4,3
8,NaN,59.37,57.10,61.40,5,4,4,1,2,3,...,1,2,5,5,2,5,4,3,4,5
9,NaN,66.93,49.10,68.20,3,4,3,3,4,3,...,4,4,1,5,3,3,2,4,4,5


In [4]:
import sys
import os
sys.path.append(os.path.abspath("../teambuilding"))
import pandas as pd
import numpy as np
import itertools
from cal import calculate_weighted_scores


objectives = okr_df['Objective']
score=okr_df['Objective Score']
posted_OKR=okr_df['OKR_NUM']

def generate_combinations_3d(data, num_parts=5):
    # Convert data to numpy array for easier manipulation
    data_values = data.values
    
    # Split data into equal parts
    part_size = len(data_values) // num_parts
    parts = [data_values[i * part_size:(i + 1) * part_size] for i in range(num_parts)]
    
    # Generate all possible combinations from the parts (row-wise combinations)
    combinations = list(itertools.product(*parts))
    
    # Convert combinations to a 3D numpy array
    combinations_3d = np.array(combinations)
    
    return combinations_3d


all_data_f = []

for i, n_okr in enumerate(objectives):
    # Step 2: cal.py의 calculate_weighted_scores 함수 사용하여 weighted_sums 계산
    weighted_sums = calculate_weighted_scores(n_okr)

    # Step 3: weighted_sums에서 두 번째 값을 추출
    weighted_values = [value[1] for value in weighted_sums]

    # Step 4: data의 첫 번째 열(0열)에 weighted_values 추가
    weighted_array = np.array(weighted_values)

   
    #
    if data.shape[0] == len(weighted_values):
        data.iloc[:, 0] = weighted_array  # pandas의 iloc 사용하여 첫 번째 열에 할당
        data['label']=score[i] #label 추가
        data["posted"]=posted_OKR[i] #OKR_NUM
    else:
        print(f"Objective {i+1}: 샘플 수가 일치하지 않습니다.")
    
    # Generate combinations based on the updated data
    data_3d = generate_combinations_3d(data.iloc[:, :], num_parts=5)

    # data_f 계산
    data_f = np.concatenate((data_3d[:,:,0:1], data_3d[:,:,4:]), axis=2)

    # data_f를 dim=0에서 쌓기 위해 리스트에 저장
    all_data_f.append(data_f)

# Step 5: dim=0에서 모든 data_f 연결
final_data_f = np.concatenate(all_data_f, axis=0)


print(final_data_f.shape)
print(final_data_f)

#np.save('sample.npy', final_data_f)


[(1, 19.46755132727325), (2, 33.1308916982015), (3, 21.547557955284912), (4, 21.2486160081625), (5, 31.560671019256116), (6, 17.605531924764318), (7, 22.384690749943257), (8, 23.471902757038674), (9, 22.774176270167032), (10, 18.856560865193604), (11, 25.031140847504144), (12, 17.147480913003285), (13, 16.220448183914026), (14, 29.94813720514377), (15, 17.552213602562745), (16, 29.9273800316453), (17, 28.244256141682467), (18, 23.03077177445094), (19, 20.40224830776453), (20, 28.030683135737974), (21, 30.655623489518963), (22, 22.644303645193578), (23, 25.245462450385094), (24, 20.42351318448782), (25, 30.684538121223454), (26, 19.17668210670352), (27, 26.604240636825562), (28, 15.446128577341634), (29, 16.707600478331248), (30, 24.36310147841772), (31, 31.761196468671162), (32, 10.364732544968525), (33, 27.10890780856212), (34, 27.1246503889064), (35, 24.68272417038679), (36, 21.041822373569012), (37, 26.193753044505915), (38, 28.38115129371484), (39, 15.357468132898212), (40, 18.661

KeyboardInterrupt: 